# Create Tiles for Indonesia Coastline

In [ ]:
import os

import geopandas as gpd
from odc.geo import BoundingBox

from coastlines.grids import INDONESIA_25, INDONESIA_CRS

os.environ["USE_PYGEOS"] = "0"

## Shows which coastlines.grids library is imported

It looks like the imported libraries are not from this repository, so we need to define the grid in this notebook and take some parameters from the existing grids.
First shows grid parameters.

In [ ]:
print('INDONESIA_25:', INDONESIA_25, INDONESIA_25.origin)
print('INDONESIA_CRS:', INDONESIA_CRS)

## Create Indonesia 25 km tiles

In [ ]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(94, -11, 142, 6, crs='EPSG:4326').to_crs(INDONESIA_25.crs)
geom = INDONESIA_25.geojson(bbox=bounds)
tiles_projected_25 = gpd.GeoDataFrame.from_features(geom, crs='EPSG:4326').to_crs(
    INDONESIA_25.crs
)

## Indonesia coast tiles

Read Indonesia's 5K coastline

In [ ]:
idn_coastline = gpd.read_file('../data/raw/GARISPANTAI_5K_SEPT2024.gdb')

In [ ]:
if tiles_projected_25.crs != idn_coastline.crs:
    print(f'Reproject tiles to {idn_coastline.crs}') 
    tiles_idn = tiles_projected_25.to_crs(crs=str(idn_coastline.crs).upper())
else:
    print('Same CRS')
    tiles_idn = tiles_projected_25

Perform spatial join: tiles that intersect any coastline segment. Tiles id column merged into the table. 

In [ ]:
# Perform spatial join: tiles that intersect any coastline segment
tiles_idn_coast = gpd.sjoin(tiles_idn, idn_coastline, predicate="intersects", how="inner")
tiles_idn_coast.head(3)

Keep unique tiles only. Removing redundant tiles and unnecessary columns.

In [ ]:
# Keep unique tiles only
tiles_idn_coast = tiles_idn.loc[tiles_idn.index.isin(tiles_idn_coast.index)]
tiles_idn_coast.head(3)

Filtered tiles based on coastline data.

In [ ]:
tiles_idn_coast.explore()

In [ ]:
print(len(tiles_idn_coast))

In [ ]:
# Export the results as GeoJSON
tiles_idn_coast = tiles_idn_coast.rename(columns={'idx': 'id'})
tiles_idn_coast.to_crs('EPSG:4326').to_file(
    '../data/raw/indonesia_25km_tiles_coast.geojson', driver='GeoJSON'
)